This is a classification example to show how to use Oboe for training and testing, in the context of AutoML, i.e., do model selection on the training set and then evaluate the performance of the selected model on the test set.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#Oboe modules; this will be simplified when Oboe becomes pip installable
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [11]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=10, verbose=False)

In [12]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [13]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.05087719298245613
elapsed time: 7.21905517578125


In [14]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'kSVM': [{'C': 0.25, 'kernel': 'poly', 'coef0': 10}],
  'AB': [{'n_estimators': 100, 'learning_rate': 3}],
  'GBT': [{'learning_rate': 0.001, 'max_depth': 3, 'max_features': 'log2'}]}}

# Example 2: build an ensemble of models

In [15]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [16]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [17]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [18]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [19]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [20]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))
print("individual accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

prediction error: 0.025438596491228094
elapsed time: 7.217111110687256
individual accuracies of selected models: [0.025438596491228094, 0.025438596491228094, 0.05087719298245613]


In [21]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'KNN': [{'n_neighbors': 11, 'p': 1},
   {'n_neighbors': 13, 'p': 1},
   {'n_neighbors': 11, 'p': 2}]}}

# Example 3: just select a collection of promising models without building an ensemble afterwards

In [22]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [23]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [24]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': False,
}

In [25]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [26]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [27]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
 
print("elapsed time: {}".format(elapsed_time))
print("accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

elapsed time: 5.099339008331299
accuracies of selected models: [0.0, 0.0, 0.0, 0.0, 0.0, 0.025438596491228094, 0.0, 0.025438596491228094, 0.025438596491228094, 0.025438596491228094, 0.025438596491228094, 0.025438596491228094, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.025438596491228094, 0.0, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.02348484848484848, 0.05087719298245613, 0.0, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.02348484848484848, 0.05087719298245613, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05087719298245613, 0.05087719298245613, 0.025438596491228094, 0.0, 0.025438596491228094, 0.05087719298245613, 0.025438596491228094, 0.025438596491228094, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298245613, 0.05087719298

Note that we do not have a single accuracy value here if we do not build an ensemble, instead, we just have a collection of fitted models with individual accuracies reported.

In [28]:
# get names of the selected machine learning models
m.get_models()

{'KNN': [{'n_neighbors': 1, 'p': 2},
  {'n_neighbors': 1, 'p': 2},
  {'n_neighbors': 1, 'p': 1},
  {'n_neighbors': 3, 'p': 1},
  {'n_neighbors': 3, 'p': 2},
  {'n_neighbors': 5, 'p': 1},
  {'n_neighbors': 5, 'p': 2},
  {'n_neighbors': 7, 'p': 1},
  {'n_neighbors': 7, 'p': 2},
  {'n_neighbors': 9, 'p': 1},
  {'n_neighbors': 9, 'p': 2},
  {'n_neighbors': 11, 'p': 1},
  {'n_neighbors': 11, 'p': 2},
  {'n_neighbors': 13, 'p': 1},
  {'n_neighbors': 13, 'p': 2},
  {'n_neighbors': 15, 'p': 1},
  {'n_neighbors': 15, 'p': 2}],
 'DT': [{'min_samples_split': 0.0001},
  {'min_samples_split': 1e-05},
  {'min_samples_split': 2},
  {'min_samples_split': 0.001},
  {'min_samples_split': 4},
  {'min_samples_split': 8},
  {'min_samples_split': 0.01},
  {'min_samples_split': 16},
  {'min_samples_split': 64}],
 'AB': [{'n_estimators': 50, 'learning_rate': 2.5},
  {'n_estimators': 50, 'learning_rate': 2},
  {'n_estimators': 50, 'learning_rate': 3},
  {'n_estimators': 100, 'learning_rate': 2},
  {'n_estimato